# Инструменты для работы с языком 

## Задача: классификация твитов по тональности

У нас есть датасет из твитов, про каждый указано, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску.

In [1]:
import pandas as pdimport numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

C:\Users\vddav\AppData\Local\Temp\ipykernel_10280\3125191635.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [3]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


In [5]:
# Векторизация текстовых данных
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(x_train)
X_test_vec = vectorizer.transform(x_test)

# Обучение классификаторов на каждой из групп токенов
def train_classifier(tokens, X_train, X_test, y_train, y_test):
    # Создание отдельных векторизаторов для каждой группы токенов
    vectorizer_tokens = CountVectorizer(vocabulary=tokens)
    X_train_tokens = vectorizer_tokens.fit_transform(X_train)
    X_test_tokens = vectorizer_tokens.transform(X_test)

    classifier = LogisticRegression()
    classifier.fit(X_train_tokens, y_train)
    y_pred = classifier.predict(X_test_tokens)
    accuracy = accuracy_score(y_test, y_pred)
    return classifier, accuracy

# Определение токенов с высокой, средней и низкой частотой
token_frequencies = X_train_vec.sum(axis=0).tolist()[0]
feature_names = vectorizer.get_feature_names_out()
high_freq_tokens = [feature_names[i] for i, freq in enumerate(token_frequencies) if freq > 100]
medium_freq_tokens = [feature_names[i] for i, freq in enumerate(token_frequencies) if 10 < freq <= 100]
low_freq_tokens = [feature_names[i] for i, freq in enumerate(token_frequencies) if freq <= 10]

classifier_high, accuracy_high = train_classifier(high_freq_tokens, x_train, x_test, y_train, y_test)
classifier_medium, accuracy_medium = train_classifier(medium_freq_tokens, x_train, x_test, y_train, y_test)
classifier_low, accuracy_low = train_classifier(low_freq_tokens, x_train, x_test, y_train, y_test)

print("Accuracy for High-Frequency Tokens:", accuracy_high)
print("Accuracy for Medium-Frequency Tokens:", accuracy_medium)
print("Accuracy for Low-Frequency Tokens:", accuracy_low)

C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracy for High-Frequency Tokens: 0.7117917790826853
Accuracy for Medium-Frequency Tokens: 0.6403216420673967
Accuracy for Low-Frequency Tokens: 0.6285245728191292


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [6]:
# Получение значимых признаков из классификаторов
significant_features_high = [vectorizer.get_feature_names_out()[i] for i in classifier_high.coef_[0].argsort()[-10:][::-1]]
significant_features_medium = [vectorizer.get_feature_names_out()[i] for i in classifier_medium.coef_[0].argsort()[-10:][::-1]]
significant_features_low = [vectorizer.get_feature_names_out()[i] for i in classifier_low.coef_[0].argsort()[-10:][::-1]]

print("Significant Features for High-Frequency Tokens:", significant_features_high, sep='\n')
print("Significant Features for Medium-Frequency Tokens:", significant_features_medium, sep='\n')
print("Significant Features for Low-Frequency Tokens:", significant_features_low, sep='\n')

Significant Features for High-Frequency Tokens:
['00lg6bsnb8', '00ngi3vmps', '00okaay', '01цитата', '0vmid1r2qo', '1ajhcakj1y', '03z65obfcx', '1wmcplml4t', '04', '01yqi8ki1v']
Significant Features for Medium-Frequency Tokens:
['0im3qastxq', '__wwonka', '0rzy39qubw', 'alinaa_bb', '__________', '_want_fly_', '_vvg', '11names', '8qhzotgetz', '0km4cqz3n9']
Significant Features for Low-Frequency Tokens:
['26uhpe3yl4', 'sun_for', 'seeker_miracle', 'feyastoporom', 'jecye7xzpf', 'emil', 'borskih', 'должностными', 'присранейшего', 'dummukycjv']


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [7]:
# Векторизация текстовых данных с разными методами
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(x_train)
X_test_count = count_vectorizer.transform(x_test)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
X_test_tfidf = tfidf_vectorizer.transform(x_test)

hashing_vectorizer = HashingVectorizer()
X_train_hashing = hashing_vectorizer.transform(x_train)
X_test_hashing = hashing_vectorizer.transform(x_test)

# Обучение и оценка моделей
def train_and_evaluate(X_train, X_test, y_train, y_test):
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report

print("Classification Report for Count Vectorizer:")
print(train_and_evaluate(X_train_count, X_test_count, y_train, y_test))

print("Classification Report for TF-IDF Vectorizer:")
print(train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test))

print("Classification Report for Hashing Vectorizer:")
print(train_and_evaluate(X_train_hashing, X_test_hashing, y_train, y_test))

Classification Report for Count Vectorizer:


C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.76      0.77      0.76     28024
    positive       0.77      0.76      0.76     28685

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

Classification Report for TF-IDF Vectorizer:
              precision    recall  f1-score   support

    negative       0.77      0.74      0.75     28024
    positive       0.75      0.78      0.77     28685

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

Classification Report for Hashing Vectorizer:


C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.76      0.73      0.74     28024
    positive       0.75      0.77      0.76     28685

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



### 3.2 с проверкой на переобучаемость

In [8]:
# Разделение данных на тренировочную, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(df.text, df.label, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Векторизация текстовых данных с разными методами
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_val_count = count_vectorizer.transform(X_val)
X_test_count = count_vectorizer.transform(X_test)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

hashing_vectorizer = HashingVectorizer()
X_train_hashing = hashing_vectorizer.transform(X_train)
X_val_hashing = hashing_vectorizer.transform(X_val)
X_test_hashing = hashing_vectorizer.transform(X_test)

# Функция для обучения и оценки модели
def train_evaluate_report(classifier, X_train, X_val, y_train, y_val):
    classifier.fit(X_train, y_train)
    
    # Оценка на валидационной выборке
    y_val_pred = classifier.predict(X_val)
    val_report = classification_report(y_val, y_val_pred)
    
    # Оценка на тренировочной выборке (для мониторинга переобучения)
    y_train_pred = classifier.predict(X_train)
    train_report = classification_report(y_train, y_train_pred)
    
    return val_report, train_report

# Обучение и оценка моделей с мониторингом переобучения
print("Classification Report for Count Vectorizer:")
val_report, train_report = train_evaluate_report(LogisticRegression(), X_train_count, X_val_count, y_train, y_val)
print("Validation Report:")
print(val_report)
print("Train Report (for monitoring overfitting):")
print(train_report)

print("Classification Report for TF-IDF Vectorizer:")
val_report, train_report = train_evaluate_report(LogisticRegression(), X_train_tfidf, X_val_tfidf, y_train, y_val)
print("Validation Report:")
print(val_report)
print("Train Report (for monitoring overfitting):")
print(train_report)

print("Classification Report for Hashing Vectorizer:")
val_report, train_report = train_evaluate_report(LogisticRegression(), X_train_hashing, X_val_hashing, y_train, y_val)
print("Validation Report:")
print(val_report)
print("Train Report (for monitoring overfitting):")
print(train_report)

Classification Report for Count Vectorizer:


C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Report:
              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     16779
    positive       0.77      0.76      0.77     17246

    accuracy                           0.76     34025
   macro avg       0.76      0.76      0.76     34025
weighted avg       0.76      0.76      0.76     34025

Train Report (for monitoring overfitting):
              precision    recall  f1-score   support

    negative       0.90      0.91      0.91     78171
    positive       0.91      0.91      0.91     80612

    accuracy                           0.91    158783
   macro avg       0.91      0.91      0.91    158783
weighted avg       0.91      0.91      0.91    158783

Classification Report for TF-IDF Vectorizer:
Validation Report:
              precision    recall  f1-score   support

    negative       0.77      0.74      0.75     16779
    positive       0.75      0.78      0.77     17246

    accuracy                           0.76     34025
  

C:\Users\vddav\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Report:
              precision    recall  f1-score   support

    negative       0.76      0.73      0.74     16779
    positive       0.75      0.78      0.76     17246

    accuracy                           0.75     34025
   macro avg       0.75      0.75      0.75     34025
weighted avg       0.75      0.75      0.75     34025

Train Report (for monitoring overfitting):
              precision    recall  f1-score   support

    negative       0.82      0.78      0.80     78171
    positive       0.80      0.83      0.81     80612

    accuracy                           0.81    158783
   macro avg       0.81      0.81      0.81    158783
weighted avg       0.81      0.81      0.81    158783

